<a href="https://colab.research.google.com/github/gzholtkevych/Lambda-Simulator/blob/main/syntax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1><b>Typeless Lambda Calculus</b></H1>

# Preparing Notebook for Using

In [ ]:
from typing import Dict, Optional, Tuple, Union, List
from typing_extensions import Self

# Syntax

## Variables

### Understanding

A variable is an atomic entity of a lambda calculus that refers to itself.<br/>
In other words,

>the value of a variable is its name.

We understand **atomicity** as

>the property of a one-to-one correspondence between variables and memory blocks allocated for them.

### Implementing

The notion is of a var implemented by the class `var`.

*Static fields*

```
__declared       is the dictionary of all declared variables;
                 the key of a variable in this dictionary is the variable name
__check_varname  is a function that takes a string representing a variable name and
                 throws an exception if the string does not match the variable
                 name format namely, an identifier starting with a lowercase letter
```

*Class methods*

```
get           returns either the variable by its name if that variable is already declared or
              returns `None` if it is not declared
all_declared  returns the list of all declared variables
```

In [ ]:
class var(str):

    __declared: Dict[str, Self] = {}  # the dictionary of declared variables
                                     # with the variable name as its key

    def __check_varname(varname: str) -> Optional[str]:
        if not isinstance(varname, str):
            return "invalid type of varname"
        if not (varname and varname.isidentifier() and varname[0].islower()):
            return "invalid format of varname"
        return None

    def __new__(cls, varname: str) -> Self:
        msg = cls.__check_varname(varname)
        if msg is not None:
            raise ValueError(f"var() error! {msg}")
        if varname not in cls.__declared:  # 'varname' is a new name
            cls.__declared[varname] = str.__new__(cls, varname)
        return cls.__declared[varname]

    def __eq__(self, another: Self) -> bool:
        if not isinstance(another, var):
            return False
        return super().__eq__(another)

    @classmethod
    def get(cls, varname: str) -> Optional[Self]:
        msg = cls.__check_varname(varname)
        if msg is not None:
            raise ValueError(f"var_get() error! {msg}")
        try:
            return cls.__declared[varname]
        except KeyError:
            return None

    @classmethod
    def all_declared(cls) -> List[Self]:
        return [cls.__declared[key] for key in cls.__declared]

## Lambda Terms

### Understanding

Lambda terms are built from variables using three constructors

- `atom`, which coerces a variable to the corresponding atomic term;
- `application`, which applies one term to another;
- `abstraction`, which declares one variable of the term as a function argument.

The following notation is usually used.

- **atom:** $\dfrac{x\text{ is a variable}}{x\text{ is a term}}$;
- **application:** $\dfrac{M\text{ is a term}\quad N\text{ is a term}}{(MN)\text{ is a term}}$;
- **abstraction:** $\dfrac{x\text{ is a variable}\quad M\text{ is a term}}{(\lambda\,x\mathop{.}M)\text{ is a term}}$.

The symbols '$\lambda$', '(', ')', and '.' are punctuation marks Lambda Calculus and they do not belong to the set of variables.

### Implementing

In [ ]:
class Term(tuple):

    ATM: int = 0
    APP: int = 1
    ABS: int = 2

    def __new__(cls, *args: Tuple) -> Self:
        if not args:
            raise ValueError("Term() error! Missing constructor argument(s)")
        if len(args) == 1 and type(args[0]) == var:  # the case of an atom
            pass
        elif len(args) == 2:  # the case of an application or an abstraction
            if type(args[0]) == Term and type(args[1]) == Term:
                pass  # the case of an application
            elif type(args[0]) == var and type(args[1]) == Term:
                pass  # the case of an abstraction
            else:
                raise TypeError(
                    "Term() error! Invalid constructor argument(s) type")
        else:
            raise ValueError("unrecognized term constructor")
        return super().__new__(cls, *args)

    @classmethod
    def atm(cls, x: Union[str, var]) -> Self:
        if type(x) == var:
            return cls(x)
        try:
            return cls(0, var(x))
        except:
            pass
        raise TypeError("atm() error! Invalid argument type")

    @classmethod
    def app(cls, t1: Self, t2: Self) -> Self:
        if type(t1) == Term and type(t2) == Term:
            return cls(t1, t2)
        raise TypeError("app() error! Invalid argument(s) type")

    @classmethod
    def abs(cls, x: Union[str, var], t: Self) -> Self:
        if type(t) == Term:
            if type(x) == var:
                return cls(x, t)
            try:
                return cls(var(x), t)
            except:
                pass
        raise TypeError("abs() error! Invalid argument(s) type")

    @property
    def kind(self) -> int:
        return (Term.ATM if len(self) == 1 else
                Term.APP if type(self[0]) == Term else
                Term.ABS)

    def __str__(self):
        if self.kind == Term.ATM:
            return self[0]
        if self.kind == Term.APP:
            return f"({self[0]} {self[1]})"
        # self.kind == Term.ABS
        return f"(λ {self[0]}.{self[1]})"

## Syntactic Characteristics of Terms

Property `size` is defined as follows

- if term $M$ matches with an atom then $\texttt{size}(M)=1$;
- if term $M$ matches with an application $(N_1\,N_2)$ then $\texttt{size}(M)=\texttt{size}(N_1)+\texttt{size}(N_2)+1$;
- if term $M$ matches with an abstraction $(\lambda\,x\mathop{.}N)$ then $\texttt{size}(M)=\texttt{size}(N)+1$.

In [ ]:
def size(t: Term) -> int:
    if type(t) != Term:
        raise TypeError("size() error!Tterm is expected")
    if t.kind == "atom":
        return 1
    if t.kind == "application":
        return size(t[0]) + size(t[1]) + 1
    # t.kind == "abstraction"
    return size(t[1]) + 1

Property `height` is defined as follows

- if term $M$ matches with an atom then $\texttt{height}(M)=1$;
- if term $M$ matches with an application $(N_1\,N_2)$ then $\texttt{size}(M)=\max(\texttt{height}(N_1),\texttt{height}(N_2))+1$;
- if term $M$ matches with an abstraction $(\lambda\,x\mathop{.}N)$ then $\texttt{height}(M)=\texttt{height}(N)+1$.

In [ ]:
def height(t: Term) -> int:
    if type(t) != Term:
        raise TypeError("size() error! Term is expected")
    if t.kind == "atom":
        return 1
    if t.kind == "application":
        return max(height(t[0]), height(t[1])) + 1
    # t.kind == "abstraction"
    return height(t[1]) + 1

Property `width` is defined as follows

- if term $M$ matches with an atom then $\texttt{width}(M)=1$;
- if term $M$ matches with an application $(N_1\,N_2)$ then $\texttt{width}(M)=\texttt{width}(N_1)+\texttt{width}(N_2)$;
- if term $M$ matches with an abstraction $(\lambda\,x\mathop{.}N)$ then $\texttt{width}(M)=\texttt{width}(N)$.

In [ ]:
def width(t: Term) -> int:
    if type(t) != Term:
        raise TypeError("width() error! Term is expected")
    if t.kind == "atom":
        return 1
    if t.kind == "application":
        return width(t[0]) + width(t[1])
    # t.kind == "abstraction"
    return width(t[1])

## Variables of a Term; Fresh, Free, and Binder Variables of a Term

### Variables of a Term

A variable $x$ is a ***variable of a term*** $M$ if

- $M=x$;
- $x$ is a variable of $N$ or $x$ is a variable of $K$ whenever $M=(N\,K)$;
- $x=y$ or $x$ is a variable of $N$ whwnever $M=(\lambda\ y\mathop{.}N)$.

In [ ]:
def isvar(x: var, M: Term) -> bool:
    """
    The function determines whether a variable is a variable of a term.

    Arguments:
        x: a variable
        M: a term
    Returns
        True:   if x is a variable of M
        False:  if x is not a variable of M
    """
    pass  # realize this function!

### Fresh Variables of a Term

A variable $x$ is called a ***fresh*** variable of a term $M$ if

- $x\neq y$ whenever $M=y$;
- $x$ is a fresh variable of both $N$ and $K$ whenever $M=(N\,K)$;
- $x\neq y$ and $x$ is a fresh variable of $N$ whenever $M=(\lambda\ x\mathop{.}N)$.


In [ ]:
def isfresh(x: var, M: Term) -> bool:
    """
    The function determines whether a variable is a fresh variable of a term.

    Arguments:
        x: a variable
        M: a term
    Returns
        True:  if x is a fresh variable of M
        False: if x is not a fresh variable of M
    """
    pass  # realize this function!

In [ ]:
def get_fresh_variable(*args: Tuple[Term, ...]) -> var:
    """
    The function returns some fresh variable of a finite set of terms.

    Arguments:
        args: a tuple of terms
    Returns
        a variable that is a fresh variable of each term in args
    """
    pass  # realize this function!

### Free Variables of a Term

A variable $x$ is a ***free*** variable of a term $M$ if

- $x=y$ whenever $M=y$;
- $x$ is a free variable of $N$ or $x$ is a free variable of $K$ whenever $M=(N\,K)$;
- $x\neq y$ and $x$ is a free variable of $N$ whenever $M=(\lambda\,y\mathop{.}N)$.

In [ ]:
def isfree(x: var, M: Term) -> bool:
    """
    The function detrmines whether a variable is a free variable of a term.

    Arguments:
        x: a variable
        M: a term
    Returns
        True:  if x is a free variable of M
        False: if x is not a free variable of M
    """
    pass  # realize this function!

### Binder Variables of a Term

A variable $x$ is a ***binder*** variable of a term $M$ if $M\neq y$ for any variable $y$ and

- $x$ is a binder variable of $N$ or $x$ is a binder variable of $K$ whenever $M=(N\,K)$;
- $x=y$ or $x$ is a binder variable of $N$ whwnever $M=(\lambda\ y\mathop{.}N)$.


In [ ]:
def isbinder(x: var, M: Term) -> bool:
    """
    The function detrmines whether a variable is a binder variable of a term.

    Arguments:
        x: a variable
        M: a term
    Returns
        True:  if x is a binder variable of M
        False: if x is not a binder variable of M
    """
    pass  # realize this function!

## Term Structure

In [ ]:
def subterm(t: Term, ref: str) -> Optional[Term]:
    if type(t) != Term:
        raise TypeError("subterm() error! Bad type of term")
    if type(ref) != str:
        raise TypeError("subterm() error! Bad type of reference")
    if ref and not all(c == '0' or c == '1' for c in ref):
        raise ValueError("subterm() error! Bad value of reference")
    if ref == "":
        return t
    if t.kind == Term.ATM:
        return None
    href, tref = ref[0], ref[1 :]
    if t.kind == Term.APP:
        if href == '0':
            return subterm(t[0], tref)
        else:
            return subterm(t[1], tref)
    # t.kind == Term.ABS
    if href == '0':
        return subterm(t[1], tref)
    return None

A term is called a ***redex*** if it is $((\lambda\,x\mathop{.}M) N)$.

In [ ]:
def isredex(t: Term) -> bool:
    return t.kind == Term.APP and t[0].kind == Term.ABS

# Transformations

## Beta-Reduction